# Init

In [6]:
%matplotlib inline

import csv
import itertools
import math
import matplotlib
import time
import logging
import sys
import os
import random
import warnings

import gensim

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

from tqdm import tqdm_notebook as tqdm
from collections import Counter, defaultdict

from sklearn import random_projection
from sklearn.manifold import TSNE
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix, spdiags
from scipy.io import loadmat, savemat
from sklearn.metrics import f1_score
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize, MultiLabelBinarizer

In [7]:
from sklearn import preprocessing

In [8]:
from scoring import scoring

In [9]:
# to ignore sklearn warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

## CiteSeer

In [11]:
citeseer = loadmat('example_graphs/citeseer.mat')
citeseer_labels = citeseer['group']
citeseer_A = citeseer['network']
N = citeseer_A.shape[0]

In [12]:
citeseer_A

<3312x3312 sparse matrix of type '<class 'numpy.int64'>'
	with 9196 stored elements in Compressed Sparse Column format>

In [13]:
N

3312

# Random Projection

In [14]:
# projection method: choose from Gaussian and Sparse
# input matrix: choose from adjacency and transition matrix
# alpha adjusts the weighting of nodes according to their degree
def randne_projection(A, q=3, dim=128, projection_method='gaussian', input_matrix='adj', alpha=None):
    assert input_matrix == 'adj' or input_matrix == 'trans'
    assert projection_method == 'gaussian' or projection_method == 'sparse'
    
    if input_matrix == 'adj':
        M = A
    else:
        N = A.shape[0]
        normalizer = spdiags(np.squeeze(1.0 / csc_matrix.sum(A, axis=1) ), 0, N, N)
        M = normalizer @ A
    # Gaussian projection matrix
    if projection_method == 'gaussian':
        transformer = random_projection.GaussianRandomProjection(n_components=dim, random_state=42)
    # Sparse projection matrix
    else:
        transformer = random_projection.SparseRandomProjection(n_components=dim, random_state=42)
    Y = transformer.fit(M)
    # Random projection for A
    if alpha is not None:
        Y.components_ = Y.components_ @ spdiags( \
                        np.squeeze(np.power(csc_matrix.sum(A, axis=1), alpha)), 0, N, N)
    cur_U = transformer.transform(M)
    U_list = [cur_U]
    
    for i in range(2, q + 1):
        cur_U = M @ cur_U
        U_list.append(cur_U)
    return U_list

In [15]:
# When weights is None, concatenate instead of linearly combines the embeddings from different powers of A
def randne_merge(U_list, weights, normalization=False):
    dense_U_list = [_U.todense() for _U in U_list] if type(U_list[0]) == csc_matrix else U_list
    _U_list = [normalize(_U, norm='l2', axis=1) for _U in dense_U_list] if normalization else dense_U_list

    if weights is None:
        return np.concatenate(_U_list, axis=1)
    U = np.zeros_like(_U_list[0])
    for cur_U, weight in zip(_U_list, weights):
        U += cur_U * weight
    U = preprocessing.scale(U.todense())
    # U = normalize(U.todense(), norm='l2', axis=1)
    return U

In [16]:
# A is always the adjacency matrix
# the choice between adj matrix and trans matrix is decided in the conf
def randne_wrapper(A, conf):
    U_list = randne_projection(A,
                               q=len(conf['weights']),
                               dim=conf['dim'],
                               projection_method=conf['projection_method'],
                               input_matrix=conf['input_matrix'],
                               alpha=conf['alpha'],
    )
    U = randne_merge(U_list, conf['weights'], conf['normalization'])
    return U

# Configuration for Easy Grid Search

In [17]:
def get_emb_filename(prefix, conf):
    return prefix + '-dim=' + str(conf['dim']) + ',projection_method=' + conf['projection_method'] \
        + ',input_matrix=' + conf['input_matrix'] + ',normalization=' + str(conf['normalization']) \
        + ',weights=' + (','.join(map(str, conf['weights'])) if conf['weights'] is not None else 'None') \
        + ',alpha=' + (str(conf['alpha']) if 'alpha' in conf else '') \
        + ',C=' + (str(conf['C']) if 'alpha' in conf else '1.0') \
        + '.mat'

# Grid Search for Degree Normalization

In [46]:
keys = ['projection_method', 'input_matrix', 'weights', 'normalization', 'dim', 'alpha', 'C']

all_dims = [512]
all_alphas = [0.1, 0.5, 0.0, -0.1]
# all_alphas = [-0.75]

all_alpha_conf_raw = itertools.product(
    # projection method
    ['sparse'],
    # input matrix
    ['trans'],
    # weights
    [
        [0.0, 0.0, 1.0, 2.0, 0, 0],
    ],
    # normalization
    [True],
    # dimensionality
    all_dims,
    # alpha
    all_alphas,
    # C
    # [0.1, 0.5, 1.0]
    [1.0]
)
all_alpha_conf = [{key: val for key, val in zip(keys, conf)} for conf in all_alpha_conf_raw]
len(all_alpha_conf)

4

In [47]:
all_alpha_conf

[{'projection_method': 'sparse',
  'input_matrix': 'trans',
  'weights': [0.0, 0.0, 1.0, 2.0, 0, 0],
  'normalization': True,
  'dim': 512,
  'alpha': 0.1,
  'C': 1.0},
 {'projection_method': 'sparse',
  'input_matrix': 'trans',
  'weights': [0.0, 0.0, 1.0, 2.0, 0, 0],
  'normalization': True,
  'dim': 512,
  'alpha': 0.5,
  'C': 1.0},
 {'projection_method': 'sparse',
  'input_matrix': 'trans',
  'weights': [0.0, 0.0, 1.0, 2.0, 0, 0],
  'normalization': True,
  'dim': 512,
  'alpha': 0.0,
  'C': 1.0},
 {'projection_method': 'sparse',
  'input_matrix': 'trans',
  'weights': [0.0, 0.0, 1.0, 2.0, 0, 0],
  'normalization': True,
  'dim': 512,
  'alpha': -0.1,
  'C': 1.0}]

Start grid search:

In [48]:
%%time
df = pd.DataFrame()

prefix = 'result/citeseer'
for conf in all_alpha_conf:
    # print (conf)
    emb_filename = get_emb_filename(prefix, conf)
    print (emb_filename)
    # first check if this file already exists
    path = Path(emb_filename)
    if not path.is_file():
        U = randne_wrapper(citeseer_A, conf)
        savemat(emb_filename, {'emb': U})
    else:
        print ('File %s already exists, skipped.' % emb_filename)
    f1_scores = scoring(
        [
            "--emb", emb_filename,
            "--network","example_graphs/citeseer.mat",
            "--num-shuffles", "5",
            "--debug",
            "--C", str(conf['C']),
            "--training-percents",
        ] + 
        [str(p) for p in range(1, 2)]
        ,
        )
    f1_scores = f1_scores[0]
    # see https://stackoverflow.com/questions/38987/how-to-merge-two-dictionaries-in-a-single-expression
    df = df.append(pd.Series({**conf, **f1_scores}), ignore_index=True)
    df.to_csv('result/citeseer-scores.txt', sep='\t', index=False, header=True)

df

result/citeseer-dim=512,projection_method=sparse,input_matrix=trans,normalization=True,weights=0.0,0.0,1.0,2.0,0,0,alpha=0.1,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 512
-------------------
Train percent: 0.01
Average score: {'micro': 0.3138761817627326, 'macro': 0.2983296923409182}
-------------------
result/citeseer-dim=512,projection_method=sparse,input_matrix=trans,normalization=True,weights=0.0,0.0,1.0,2.0,0,0,alpha=0.5,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 512
-------------------
Train percent: 0.01
Average score: {'micro': 0.3504727050930162, 'macro': 0.328236409787112}
-------------------
result/citeseer-dim=512,projection_method=sparse,input_matrix=trans,normalization=True,weights=0.0,0.0,1.0,2.0,0,0,alpha=0.0,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 512
-------------------
Train percent: 0.01
Average score: {'micro': 0.2900274473924977, 'macro': 0.2688619

# Intelligent Hyperparameter Search

In [35]:
import optuna

In [63]:
%%time
order_range = 3
def objective(trial):

    # Invoke suggest methods of a Trial object to generate hyperparameters.
    weights = [trial.suggest_loguniform('weight' + str(order), 1.0, 64.0) for order in range(order_range)]
    conf = {
        'projection_method': 'sparse',
        'input_matrix': 'trans',
        'weights': [1.0] + weights,
        'normalization': True,
        'dim': 128,
        'alpha': 0,
        'C': 1.0
    }
    emb_filename = get_emb_filename(prefix, conf)
    print (emb_filename)
    # first check if this file already exists
    path = Path(emb_filename)
    if not path.is_file():
        U = randne_wrapper(citeseer_A, conf)
        savemat(emb_filename, {'emb': U})
    else:
        print ('File %s already exists, skipped.' % emb_filename)
    f1_scores = scoring(
        [
            "--emb", emb_filename,
            "--network","example_graphs/citeseer.mat",
            "--num-shuffles", "5",
            "--debug",
            "--C", str(conf['C']),
            "--training-percents",
            "1",
        ],
    )
    # there should only be one entry here
    return -f1_scores[0]['macro']

study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=100)  # Invoke optimization of the objective function.

result/citeseer-dim=128,projection_method=sparse,input_matrix=trans,normalization=True,weights=1.0,5.289138527533144,2.4559455837455078,1.4551785241567021,alpha=0,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 128
-------------------
Train percent: 0.01
Average score: {'micro': 0.25227203415675514, 'macro': 0.2272149809153879}
-------------------


[I 2019-03-22 19:41:27,145] Finished trial#0 resulted in value: -0.2272149809153879. Current best value is -0.2272149809153879 with parameters: {'weight0': 5.289138527533144, 'weight1': 2.4559455837455078, 'weight2': 1.4551785241567021}.


result/citeseer-dim=128,projection_method=sparse,input_matrix=trans,normalization=True,weights=1.0,56.08826672013705,56.128165696706915,32.84849391545937,alpha=0,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 128
-------------------
Train percent: 0.01
Average score: {'micro': 0.2653247941445563, 'macro': 0.2404721647990399}
-------------------


[I 2019-03-22 19:41:27,430] Finished trial#1 resulted in value: -0.2404721647990399. Current best value is -0.2404721647990399 with parameters: {'weight0': 56.08826672013705, 'weight1': 56.128165696706915, 'weight2': 32.84849391545937}.


result/citeseer-dim=128,projection_method=sparse,input_matrix=trans,normalization=True,weights=1.0,3.8899222757951155,19.271430136041285,17.145821323163478,alpha=0,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 128
-------------------
Train percent: 0.01
Average score: {'micro': 0.28032936870997255, 'macro': 0.24952286389315143}
-------------------


...

result/citeseer-dim=128,projection_method=sparse,input_matrix=trans,normalization=True,weights=1.0,4.938942240812036,4.084894284007911,45.36831141286988,alpha=0,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 128
-------------------
Train percent: 0.01
Average score: {'micro': 0.2616651418115279, 'macro': 0.23463628877845868}
-------------------


[I 2019-03-22 19:42:00,123] Finished trial#98 resulted in value: -0.23463628877845868. Current best value is -0.2641729807521684 with parameters: {'weight0': 2.608108225253218, 'weight1': 5.1162063998561935, 'weight2': 18.659171473524886}.


result/citeseer-dim=128,projection_method=sparse,input_matrix=trans,normalization=True,weights=1.0,3.672983560184079,9.363571204278164,22.043965612979317,alpha=0,C=1.0.mat
[0.01]
Train percent: 0.01
Results, using embeddings of dimensionality 128
-------------------
Train percent: 0.01
Average score: {'micro': 0.2694724001219885, 'macro': 0.2381166175282427}
-------------------


[I 2019-03-22 19:42:00,439] Finished trial#99 resulted in value: -0.2381166175282427. Current best value is -0.2641729807521684 with parameters: {'weight0': 2.608108225253218, 'weight1': 5.1162063998561935, 'weight2': 18.659171473524886}.


CPU times: user 11min 10s, sys: 1.71 s, total: 11min 12s
Wall time: 33.6 s


In [64]:
study.best_params

{'weight0': 2.608108225253218,
 'weight1': 5.1162063998561935,
 'weight2': 18.659171473524886}

In [67]:
conf = {
        'projection_method': 'sparse',
        'input_matrix': 'trans',
        'weights': [1.0] + [study.best_params[key] for key in ['weight0', 'weight1', 'weight2'] ],
        'normalization': True,
        'dim': 512,
        'alpha': 0,
        'C': 1.0
    }
emb_filename = get_emb_filename(prefix, conf)
print (emb_filename)
# first check if this file already exists
path = Path(emb_filename)
if not path.is_file():
    U = randne_wrapper(citeseer_A, conf)
    savemat(emb_filename, {'emb': U})
else:
    print ('File %s already exists, skipped.' % emb_filename)
f1_scores = scoring(
    [
        "--emb", emb_filename,
        "--network","example_graphs/citeseer.mat",
        "--num-shuffles", "5",
        "--debug",
        "--C", str(conf['C']),
        "--training-percents",
    ] + [str(p) for p in range(1, 10)],
)
f1_scores

result/citeseer-dim=512,projection_method=sparse,input_matrix=trans,normalization=True,weights=1.0,2.608108225253218,5.1162063998561935,18.659171473524886,alpha=0,C=1.0.mat
[0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]
Train percent: 0.01
Train percent: 0.02
Train percent: 0.03
Train percent: 0.04
Train percent: 0.05
Train percent: 0.06
Train percent: 0.07
Train percent: 0.08
Train percent: 0.09
Results, using embeddings of dimensionality 512
-------------------
Train percent: 0.01
Average score: {'micro': 0.291491308325709, 'macro': 0.2705545220098186}
-------------------
Train percent: 0.02
Average score: {'micro': 0.34861367837338264, 'macro': 0.32574329636313737}
-------------------
Train percent: 0.03
Average score: {'micro': 0.37640834111422344, 'macro': 0.3515038350653313}
-------------------
Train percent: 0.04
Average score: {'micro': 0.4034591194968554, 'macro': 0.377041305424901}
-------------------
Train percent: 0.05
Average score: {'micro': 0.4215443279313632, 'm

[defaultdict(float,
             {'micro': 0.291491308325709,
              'macro': 0.2705545220098186,
              'training': 0.01}),
 defaultdict(float,
             {'micro': 0.34861367837338264,
              'macro': 0.32574329636313737,
              'training': 0.02}),
 defaultdict(float,
             {'micro': 0.37640834111422344,
              'macro': 0.3515038350653313,
              'training': 0.03}),
 defaultdict(float,
             {'micro': 0.4034591194968554,
              'macro': 0.377041305424901,
              'training': 0.04}),
 defaultdict(float,
             {'micro': 0.4215443279313632,
              'macro': 0.39453009016821305,
              'training': 0.05}),
 defaultdict(float,
             {'micro': 0.42780989081567117,
              'macro': 0.3991497994764396,
              'training': 0.06}),
 defaultdict(float,
             {'micro': 0.4341447581953911,
              'macro': 0.40490229837413033,
              'training': 0.07}),
 defaultdict(flo